<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import project files files from github 


In [ ]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

fatal: destination path 'IncrementalLearning' already exists and is not an empty directory.


import libraries


In [ ]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb

# project classes --------------------------------------------------------------
from IncrementalLearning.cifar100 import ilCIFAR100
from baselines.resnet import resnet32
from info_recorder.info_log import InfoLog
from models_benchmark.benchmark import Benchmark

importing Jupyter notebook from /content/IncrementalLearning/cifar100.ipynb


Initial setup

In [ ]:
# Define params

NUM_RUN = 2
SEEDS = [1,2,3]
NUM_EPOCHS = 5
BATCH_SIZE = 64
CLASSES_PER_BATCH = 10
DEVICE = 'cuda'

WEIGHT_DECAY = 1e-5
LR = 2
MOMENTUM = 0.9
MILESTONES = [49, 63]
GAMMA = 0.2

SAVINGS_DIR = "savings"
# Create savings folder
try:
    os.mkdir(SAVINGS_DIR)
except FileExistsError:
    pass

Define transforms

In [ ]:
# Define transformations for training
train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define transformations for evaluation
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

Prepare dataloders for each train, val test subset of the original dataset

In [ ]:
# dataloaders dict initialization
dataloaders = {run:{batch:{'train':None, 'val':None, 'test':None} for batch in range(CLASSES_PER_BATCH)} for run in range(NUM_RUN)}

# make dataloaders
for run in range(NUM_RUN):
    
    # download dataset only on the first run
    download = True
    if run != 0:
        download = False
    
    original_training_set = ilCIFAR100(CLASSES_PER_BATCH, seed=SEEDS[run])
    original_test_set = ilCIFAR100(CLASSES_PER_BATCH, train=False, seed=SEEDS[run])
    
    train_indices, val_indices = original_training_set.get_train_val(0.1)
    test_indices = original_training_set.getbatches()

    for class_batch in range(CLASSES_PER_BATCH):
        
        # test indices are the uninion of test indices of batches seen up to now
        test_indices = []
        for i in range(class_batch+1):
            test_indices += original_test_set.batches[i]

        # make subsets
        train_set = Subset(dataset=original_training_set, indices=train_indices[class_batch])
        val_set = Subset(dataset=original_training_set, indices=val_indices[class_batch])
        test_set = Subset(dataset=original_test_set, indices=test_indices[class_batch])

        # make dataloaders
        train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
        test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

        # add dataloaders to the dataloaders dict
        dataloaders[run][class_batch]['train'] = train_dataloader
        dataloaders[run][class_batch]['val'] = val_dataloader
        dataloaders[run][class_batch]['test'] = test_dataloader

Files already downloaded and verified
[17, 72, 97, 8, 32, 15, 63, 57, 60, 83, 48, 26, 12, 62, 3, 49, 55, 77, 0, 92, 34, 29, 75, 13, 40, 85, 2, 74, 69, 1, 89, 27, 54, 98, 28, 56, 93, 35, 14, 22, 61, 43, 59, 71, 78, 18, 70, 88, 86, 41, 6, 11, 82, 46, 67, 7, 21, 95, 68, 42, 87, 19, 45, 31, 47, 25, 73, 30, 94, 23, 79, 39, 76, 58, 5, 64, 99, 91, 52, 24, 33, 80, 16, 66, 90, 96, 50, 84, 36, 44, 81, 10, 9, 38, 20, 4, 51, 65, 37, 53]
Files already downloaded and verified
[17, 72, 97, 8, 32, 15, 63, 57, 60, 83, 48, 26, 12, 62, 3, 49, 55, 77, 0, 92, 34, 29, 75, 13, 40, 85, 2, 74, 69, 1, 89, 27, 54, 98, 28, 56, 93, 35, 14, 22, 61, 43, 59, 71, 78, 18, 70, 88, 86, 41, 6, 11, 82, 46, 67, 7, 21, 95, 68, 42, 87, 19, 45, 31, 47, 25, 73, 30, 94, 23, 79, 39, 76, 58, 5, 64, 99, 91, 52, 24, 33, 80, 16, 66, 90, 96, 50, 84, 36, 44, 81, 10, 9, 38, 20, 4, 51, 65, 37, 53]


TypeError: ignored

FINE TUNING

In [ ]:
for run in range(NUM_RUN):
    benchmark = Benchmark(
        num_epochs=NUM_EPOCHS, 
        dataloaders=dataloaders,
        batch_size=BATCH_SIZE,
        saving_folder="savings/fine_tuning")
    
    benchmark.set_criterion(nn.CrossEntropyLoss())
    benchmark.set_infoLog(InfoLog, run)
    benchmark.set_model(resnet32())

    for class_batch in range(NUM_CLASS_BATCH):
        parameters_to_optimize = benchmark.model.parameters()
        benchmark.set_optimizer(optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY))
        benchmark.set_scheduler(optim.lr_scheduler.MultiStepLR(benchmark.optimizer, milestones=MILESTONES, gamma=GAMMA))

        benchmark.do_class_batch(run, class_batch)

        benchmark.model.add_nodes(10) # forse è meglio metterlo all'inizio

RUN 0 ------------------------------------------------------------------------------------

	CLASS BATCH 0

	epoch  0:    train_acc: 0.096  train_loss: 2.991  val_acc: 0.100  val_loss: 2.513
	epoch  1:    train_acc: 0.106  train_loss: 2.386  val_acc: 0.100  val_loss: 2.401
	epoch  2:    train_acc: 0.103  train_loss: 2.385  val_acc: 0.098  val_loss: 2.365
	epoch  3:    train_acc: 0.092  train_loss: 2.387  val_acc: 0.100  val_loss: 2.367
	epoch  4:    train_acc: 0.099  train_loss: 2.389  val_acc: 0.100  val_loss: 2.379

	BEST RESULTS:	val_loss: 2.365,  val_acc: 0.098,  epoch: 2
	TEST ACC: 0.101


	CLASS BATCH 1

	epoch  0:    train_acc: 0.096  train_loss: 2.509  val_acc: 0.100  val_loss: 2.409
	epoch  1:    train_acc: 0.102  train_loss: 2.396  val_acc: 0.100  val_loss: 2.404
	epoch  2:    train_acc: 0.100  train_loss: 2.386  val_acc: 0.100  val_loss: 2.404
	epoch  3:    train_acc: 0.099  train_loss: 2.387  val_acc: 0.100  val_loss: 2.379
	epoch  4:    train_acc: 0.099  train_loss: 2.361 